In [ ]:
# Import required modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

In [ ]:
# Load data
train_data = pd.read_csv('./data/train.csv')
test_data = pd.read_csv('./data/test.csv')
train_data.head(5)

In [ ]:
# Describe numerical and categorical data
train_data.describe()
train_data.describe(include='object')

In [ ]:
# Extract group ID
train_data["GroupId"] = train_data["PassengerId"].str[:4]
test_data["GroupId"] = test_data["PassengerId"].str[:4]
train_data = train_data.drop(["PassengerId", "Name"], axis=1)

In [ ]:
# Select features and find numerical and categorical columns
X_cols = [col for col in train_data.columns.tolist() if col != "Transported"]
y_col = "Transported"
X_cols_num = train_data[X_cols].select_dtypes(include=np.number).columns.tolist()
X_cols_cat = [col for col in X_cols if col not in X_cols_num]

X_train = train_data[X_cols]
y_train = train_data[y_col]

X_test = test_data[X_cols]

In [ ]:
# Create transformer to create age group column
class AgeTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X_ = X.copy()
        X_["Age"] = pd.cut(X_train["Age"], bins=[0,20,40,60,80,100,120], labels=[1, 2, 3, 4, 5, 6])
        return X_


In [ ]:
# Create pipelines for numerical and categorical features and select model
num_pipeline = Pipeline([
    ("transformer", AgeTransformer()),
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ("num", num_pipeline, X_cols_num),
    ("cat", cat_pipeline, X_cols_cat)
])

model = RandomForestClassifier(n_estimators=100, random_state=0)

final_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("model", model)
])

In [ ]:
# Fit final pipeline using training data
final_pipeline.fit(X_train, y_train)

In [ ]:
# Make predictions on test data
predictions = final_pipeline.predict(X_test)
final_predictions = pd.DataFrame({"PassengerId": test_data["PassengerId"], "Transported": predictions})
final_predictions.head(5)

In [ ]:
# Save output
final_predictions.to_csv("./output/spaceship_titanic_predictions.csv", index=False)